
对于两个变量\(X\)和\(Y\)，它们的皮尔逊相关系数\(r_{XY}\)计算公式为：

$$
r_{XY}=\frac{\sum_{i = 1}^{n}(x_{i}-\bar{x})(y_{i}-\bar{y})}{\sqrt{\sum_{i = 1}^{n}(x_{i}-\bar{x})^{2}}\sqrt{\sum_{i = 1}^{n}(y_{i}-\bar{y})^{2}}}
$$

其中：
- \(n\)是样本数量；
- $x_{i}$是变量\(X\)的第\(i\)个观测值；
- $y_{i}$是变量\(Y\)的第\(i\)个观测值；
- $\bar{x}$是变量\(X\)的均值，$$\bar{x}=\frac{1}{n}\sum_{i = 1}^{n}x_{i}$$
- $\bar{y}$是变量\(Y\)的均值，$$\bar{y}=\frac{1}{n}\sum_{i = 1}^{n}y_{i}$$

In [68]:
# dependencies
import numpy as np
import pandas as pd
from pprint import pprint
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import os

from torchvision.models import vgg13
# path of datasets
# /Users/sequel/linkcodes/datasets
# hetrec2011/lastfm
# movide32

“hetrec2011-lastfm-2k”数据集包含了来自Last.fm在线音乐系统中2K用户的社交网络、标签和音乐艺术家收听信息，其涉及的文件及其用途如下：

1. **artists.dat**：记录用户收听和标记的音乐艺术家相关信息。每行数据包含艺术家的ID、名称、Last.fm上的网址以及图片网址，如“707 Metallica http://www.last.fm/music/Metallica http://userserve-ak.last.fm/serve/252/7560709.jpg” ，为研究艺术家的基本信息提供了数据支持。
2. **tags.dat**：存放数据集中可用的标签信息。每行数据由标签ID和标签值组成，例如“1 metal”，用于标记艺术家，帮助对艺术家进行分类和特征描述。
3. **user_artists.dat**：记录每个用户收听的艺术家信息，并给出每个用户与艺术家组合的收听次数。数据格式为“userID artistID weight”，其中weight代表收听次数，如“2 51 13883” ，可用于分析用户的音乐偏好和收听行为。
4. **user_taggedartists.dat**：记录每个用户对艺术家的标签分配情况，还包含标签分配的日期信息。数据格式为“userID artistID tagID day month year”，如“2 52 13 1 4 2009” ，通过这些数据可以了解用户对不同艺术家的个性化标签标注时间，分析用户标签行为的时间规律。
5. **user_taggedartists-timestamps.dat**：同样记录用户对艺术家的标签分配，但提供的是标签分配的时间戳信息。数据格式为“userID artistID tagID timestamp”，如“2 52 13 1238536800000” ，时间戳能更精确地反映标签行为发生的时间，对于研究用户行为的时间序列特征更有价值。
6. **user_friends.dat**：记录数据库中用户之间的朋友关系。每行数据包含两个用户ID，如“2 275”，表示用户2和用户275是朋友关系，可用于研究用户的社交网络结构和社交关系对音乐偏好的影响。

In [69]:
def load_dataset(path="./hetrec2011-lastfm-2k"):
    # Load user-artist interactions
    user_artists = pd.read_csv(f"{path}/user_artists.dat", sep='\t')
    
    # Load artists data
    artists = pd.read_csv(f"{path}/artists.dat", sep='\t')
    
    return user_artists, artists

In [70]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity



# 创建一个简单的用户-物品DataFrame
data = {
    'userID': [1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4],
    'itemID': ['A', 'B', 'C', 'D', 'A', 'C', 'E', 'B', 'C', 'D', 'C', 'D'],
    'rating': [5, 3, 0, 1, 4, 0, 2, 1, 0, 5, 5, 4]
}

df = pd.DataFrame(data)

# 创建用户-物品矩阵
user_item_matrix = df.pivot(index='userID', columns='itemID', values='rating').fillna(0)

print("用户-物品矩阵:")
print(user_item_matrix)

# 计算余弦相似度
similarity_matrix = pd.DataFrame(
    cosine_similarity(user_item_matrix),
    index=user_item_matrix.index,
    columns=user_item_matrix.index
)

print("\n余弦相似度矩阵:")
print(similarity_matrix)

# 为了更好的可读性，我们可以将结果四舍五入到小数点后三位
rounded_similarity = similarity_matrix.round(3)
print("\n四舍五入后的余弦相似度矩阵:")
print(rounded_similarity)

用户-物品矩阵:
itemID    A    B    C    D    E
userID                         
1       5.0  3.0  0.0  1.0  0.0
2       4.0  0.0  0.0  0.0  2.0
3       0.0  1.0  0.0  5.0  0.0
4       0.0  0.0  5.0  4.0  0.0

余弦相似度矩阵:
userID         1         2         3         4
userID                                        
1       1.000000  0.755929  0.265197  0.105593
2       0.755929  1.000000  0.000000  0.000000
3       0.265197  0.000000  1.000000  0.612564
4       0.105593  0.000000  0.612564  1.000000

四舍五入后的余弦相似度矩阵:
userID      1      2      3      4
userID                            
1       1.000  0.756  0.265  0.106
2       0.756  1.000  0.000  0.000
3       0.265  0.000  1.000  0.613
4       0.106  0.000  0.613  1.000


# 手动计算余弦相似度VS用封装好的方法计算

In [157]:
v1 = np.array([5, 3, 4, 3, 1])
v3 = np.array([3, 1, 3, 3, 5])

# 手动计算余弦相似度
dot = np.dot(v1, v3)
norm = np.linalg.norm(v1) * np.linalg.norm(v3)
cosine_sim_manual = dot / norm

print("手动计算的余弦相似度:", cosine_sim_manual)

# 使用 cosine_similarity 函数
# 注意：我们需要将向量重塑为 2D 数组
v1_reshaped = v1.reshape(1, -1)
v3_reshaped = v3.reshape(1, -1)
cosine_sim_func = cosine_similarity(v1_reshaped, v3_reshaped)
np_consine = np.corrcoef(v1, v3)[0][1]
print("使用np计算皮尔逊", np_consine)
print("工具计算余弦：", cosine_sim_func)
print("使用 cosine_similarity 函数计算的余弦相似度:", cosine_sim_func[0][0])

# 验证两种方法的结果是否相同
print("两种方法的结果是否相同:", np.isclose(cosine_sim_manual, cosine_sim_func[0][0]))


手动计算的余弦相似度: 0.7802595923450996
使用np计算皮尔逊 -0.47673129462279606
工具计算余弦： [[0.78025959]]
使用 cosine_similarity 函数计算的余弦相似度: 0.7802595923450996
两种方法的结果是否相同: True


# 手动计算皮尔逊VS工具计算
余弦相似度在度量文本相似度、用户相似度、物品相似度的时候都较为常用。

皮尔逊相关度，实际上也是一种余弦相似度。不过先对向量做了中心化，范围在[-1, 1]
- 相关度量的是两个变量的变化趋势是否一致，两个随机变量是不是同增同减。
- 不适合用作计算布尔值向量（0-1）之间相关度。

In [147]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
i = [483,133]
j = [216,4]

matrix = np.array([[1, 0, 0, 0],
                   [1, 0, 1, 0],
                   [5,4,3,2]])

# 计算皮尔逊相关系数
pearson_corr = np.corrcoef(matrix)

# 手动计算皮尔逊相关系数
n = len(i)
# 计算均值
mean_i = np.mean(i)
mean_j = np.mean(j)
# 计算分子
numerator = np.sum((np.array(i) - mean_i) * (np.array(j) - mean_j))
# 计算分母
denominator = np.sqrt(np.sum((np.array(i) - mean_i) ** 2)) * np.sqrt(np.sum((np.array(j) - mean_j) ** 2))
# 计算皮尔逊相关系数
manual_pearson = numerator / denominator
print("手动计算皮尔逊相关系数：", manual_pearson)

# 工具计算
cos = cosine_similarity([i, j])
pear = pearsonr(i, j)
anotherPear = np.corrcoef(i, j)[0][1]
pearson_corr = np.corrcoef(matrix)
print("矩阵的皮尔森系数")
print(pearson_corr)
print("另一种方法计算:", anotherPear)
print("余弦相似度:\n", cos)
print("工具计算皮尔逊", pear)

手动计算皮尔逊相关系数： 1.0
矩阵的皮尔森系数
[[1.         0.57735027 0.77459667]
 [0.57735027 1.         0.4472136 ]
 [0.77459667 0.4472136  1.        ]]
另一种方法计算: 1.0
余弦相似度:
 [[1.         0.96886623]
 [0.96886623 1.        ]]
工具计算皮尔逊 PearsonRResult(statistic=1.0, pvalue=1.0)


# Item-CF
## 上手例子

In [152]:
# 1. 建立数据
def loadData():
    items = {'A': {'Alice': 5.0, 'user1': 3.0, 'user2': 4.0, 'user3': 3.0, 'user4': 1.0},
             'B': {'Alice': 3.0, 'user1': 1.0, 'user2': 3.0, 'user3': 3.0, 'user4': 5.0},
             'C': {'Alice': 4.0, 'user1': 2.0, 'user2': 4.0, 'user3': 1.0, 'user4': 5.0},
             'D': {'Alice': 4.0, 'user1': 3.0, 'user2': 3.0, 'user3': 5.0, 'user4': 2.0},
             'E': {'user1': 3.0, 'user2': 5.0, 'user3': 4.0, 'user4': 1.0}
             }
    return items

item_data = loadData()

"""
 相似矩阵
     A    B    C    D    E
A  1.0  0.0  0.0  0.0  0.0
B  0.0  1.0  0.0  0.0  0.0
C  0.0  0.0  1.0  0.0  0.0
D  0.0  0.0  0.0  1.0  0.0
E  0.0  0.0  0.0  0.0  1.0

ABCDE是文档中的物品
"""
# 2. 建立物品间的相似度矩阵，行与行之间的相似度，第一行与第二行的相似度=sim[1][2] or sim[2][1]
similarity_matrix = pd.DataFrame(
    np.identity(len(item_data)),
    index=item_data.keys(),
    columns=item_data.keys(),
)

# 遍历每条物品-用户评分数据
for i1, name2rate1 in item_data.items():
    for i2, name2rate2 in item_data.items():
        if i1 == i2:
            continue
        vec1, vec2 = [], []
        for user, rating1 in name2rate1.items():
            rating2 = name2rate2.get(user, -1)
            if rating2 == -1:
                continue
            vec1.append(rating1)
            vec2.append(rating2)
        # print(vec1, vec2)
        similarity_matrix.loc[i2, i1] = np.corrcoef(vec1, vec2)[0][1]

user_item_mydf = pd.DataFrame(item_data).fillna(0)
item_similarity_matrix = cosine_similarity(user_item_mydf.T)
# 将相似度矩阵转换为DataFrame
direct_similarity_df = pd.DataFrame(item_similarity_matrix, index=user_item_mydf.columns, columns=user_item_mydf.columns)
print("未平等计算的皮尔逊:")
print(direct_similarity_df)
print("平等计算的皮尔逊:")
print(similarity_matrix)
similarity_matrix = direct_similarity_df
# ----------
#x loc : 先取行再取列
# similarity_matrix.loc['A', 'B'] = -1
# print(similarity_matrix)
# print("这样取得是行向量\n", similarity_matrix.loc['B'])
# print("这样取得是列向量\n", similarity_matrix['B'])

未平等计算的皮尔逊:
        A       B       C       D       E
A 1.00000 0.78026 0.81978 0.94337 0.75926
B 0.78026 1.00000 0.94202 0.84798 0.67320
C 0.81978 0.94202 1.00000 0.78403 0.62243
D 0.94337 0.84798 0.78403 1.00000 0.81153
E 0.75926 0.67320 0.62243 0.81153 1.00000
平等计算的皮尔逊:
         A        B        C        D        E
A  1.00000 -0.47673 -0.12309  0.53218  0.96946
B -0.47673  1.00000  0.64550 -0.31009 -0.47809
C -0.12309  0.64550  1.00000 -0.72058 -0.42762
D  0.53218 -0.31009 -0.72058  1.00000  0.58168
E  0.96946 -0.47809 -0.42762  0.58168  1.00000


In [150]:
# 平等计算 df
import numpy as np
import pandas as pd

# 1. 建立数据
def loadData():
    items = {'A': {'Alice': 5.0, 'user1': 3.0, 'user2': 4.0, 'user3': 3.0, 'user4': 1.0},
             'B': {'Alice': 3.0, 'user1': 1.0, 'user2': 3.0, 'user3': 3.0, 'user4': 5.0},
             'C': {'Alice': 4.0, 'user1': 2.0, 'user2': 4.0, 'user3': 1.0, 'user4': 5.0},
             'D': {'Alice': 4.0, 'user1': 3.0, 'user2': 3.0, 'user3': 5.0, 'user4': 2.0},
             'E': {'user1': 3.0, 'user2': 5.0, 'user3': 4.0, 'user4': 1.0}
             }
    return items

item_data = loadData()

# 将 item_data 转换为 DataFrame，并转置
item_df = pd.DataFrame(item_data).fillna(0).T

# 2. 建立物品间的相似度矩阵
similarity_matrix = pd.DataFrame(
    np.identity(len(item_df.index)),
    index=item_df.index,
    columns=item_df.index,
)

# 遍历每对物品，计算相似度
for i1 in item_df.index:
    for i2 in item_df.index:
        if i1 == i2:
            continue
        # 选择两个物品的评分，并过滤掉任意一个为0的评分
        ratings = item_df.loc[[i1, i2]]
        ratings = ratings.loc[:, (ratings.loc[i1] != 0) & (ratings.loc[i2] != 0)]

        # 如果没有共同的非零评分，跳过
        if ratings.shape[1] == 0:
            continue

        # 计算皮尔逊相关系数
        corr = np.corrcoef(ratings)[0][1]
        similarity_matrix.loc[i2, i1] = corr

print("物品间的相似度矩阵:")
print(similarity_matrix)

物品间的相似度矩阵:
         A        B        C        D        E
A  1.00000 -0.47673 -0.12309  0.53218  0.96946
B -0.47673  1.00000  0.64550 -0.31009 -0.47809
C -0.12309  0.64550  1.00000 -0.72058 -0.42762
D  0.53218 -0.31009 -0.72058  1.00000  0.58168
E  0.96946 -0.47809 -0.42762  0.58168  1.00000


np.corrcoef(vec1, vec2)：

计算两个向量 vec1 和 vec2 的皮尔逊相关系数（Pearson correlation coefficient）。
- 皮尔逊相关系数衡量的是两个变量之间的线性相关性，取值范围为 [-1, 1]，其中：
    - 1 表示完全正相关
    - -1 表示完全负相关
    - 0 表示无线性相关性

In [153]:
# 3. 从 Alice 购买过的物品中，选出与物品 E 最相似的 num 件物品
target_user = 'Alice'
target_item = 'E'
num = 2

sim_items = []
print(similarity_matrix[target_item])
# 降序、取indexABCDE这种g
sim_items_list = similarity_matrix[target_item].sort_values(ascending=False).index.tolist()
print(sim_items_list)
for item in sim_items_list:
    # 如果target_user对物品item评分过
    if target_user in item_data[item]:
        sim_items.append(item)
    if len(sim_items) == num:
        break
print(f'与物品{target_item}最相似的{num}个物品为：{sim_items}')

A   0.75926
B   0.67320
C   0.62243
D   0.81153
E   1.00000
Name: E, dtype: float64
['E', 'D', 'A', 'B', 'C']
与物品E最相似的2个物品为：['D', 'A']


In [154]:
# 4. 利用3的最近邻n个物品 预测用户 Alice 对物品 E 的评分
print(list(item_data[target_item].values()))
# 除掉alice外其他用户对物品E的评分均值
other_user2target_item_rate = np.mean(list(item_data[target_item].values()))
weighted_scores = 0.
corr_values_sum = 0.

target_item = 'E'
for item in sim_items:
    # 当前item和物品E的相似度
    corr_value = similarity_matrix[target_item][item]
    # 所有用户对当前item的评分均值
    user_mean_rating = np.mean(list(item_data[item].values()))
    # 累加【item与targetitem的相似度 * targetuser对item的偏好度】 / 累加的（相似度）= targetuser对targetitem对偏好度
    weighted_scores += corr_value * (item_data[item][target_user] - user_mean_rating)
    corr_values_sum += corr_value

target_item_pred = other_user2target_item_rate + weighted_scores / corr_values_sum
print(f'用户{target_user}对物品{target_item}的预测评分为：{target_item_pred}')

[3.0, 5.0, 4.0, 1.0]
用户Alice对物品E的预测评分为：4.430034365352817


# User-CF
## 上手例子

In [11]:
def loadData():
    users = {'Alice': {'A': 5, 'B': 3, 'C': 4, 'D': 4},
             'user1': {'A': 3, 'B': 1, 'C': 2, 'D': 3, 'E': 3},
             'user2': {'A': 4, 'B': 3, 'C': 4, 'D': 3, 'E': 5},
             'user3': {'A': 3, 'B': 3, 'C': 1, 'D': 5, 'E': 4},
             'user4': {'A': 1, 'B': 5, 'C': 5, 'D': 2, 'E': 1}
             }
    return users

user_data = loadData()
similarity_matrix = pd.DataFrame(
    np.identity(len(user_data)),
    index=user_data.keys(),
    columns=user_data.keys(),
)
'''similarity_matrix 
       Alice  user1  user2  user3  user4
Alice    1.0    0.0    0.0    0.0    0.0
user1    0.0    1.0    0.0    0.0    0.0
user2    0.0    0.0    1.0    0.0    0.0
user3    0.0    0.0    0.0    1.0    0.0
user4    0.0    0.0    0.0    0.0    1.0
'''
# 遍历每条用户-物品评分数据
for u1, items1 in user_data.items():
    for u2, items2 in user_data.items():
        if u1 == u2:
            continue
        vec1, vec2 = [], []
        for item, rating1 in items1.items():
            rating2 = items2.get(item, -1)
            if rating2 == -1:
                continue
            vec1.append(rating1)
            vec2.append(rating2)
        # 计算不同用户之间的皮尔逊相关系数
        similarity_matrix.loc[u2, u1] = np.corrcoef(vec1, vec2)[0][1]
print(similarity_matrix)

          Alice     user1     user2     user3     user4
Alice  1.000000  0.852803  0.707107  0.000000 -0.792118
user1  0.852803  1.000000  0.467707  0.489956 -0.900149
user2  0.707107  0.467707  1.000000 -0.161165 -0.466569
user3  0.000000  0.489956 -0.161165  1.000000 -0.641503
user4 -0.792118 -0.900149 -0.466569 -0.641503  1.000000


In [12]:
target_user = 'Alice'
num = 2
# 由于最相似的用户为自己，去除本身
copy = similarity_matrix.copy()
sim_users = copy[target_user].sort_values(ascending=False)[1:num+1].index.tolist()
print(f'与用户{target_user}最相似的{num}个用户为：{sim_users}')
print(similarity_matrix)

与用户Alice最相似的2个用户为：['user1', 'user2']
          Alice     user1     user2     user3     user4
Alice  1.000000  0.852803  0.707107  0.000000 -0.792118
user1  0.852803  1.000000  0.467707  0.489956 -0.900149
user2  0.707107  0.467707  1.000000 -0.161165 -0.466569
user3  0.000000  0.489956 -0.161165  1.000000 -0.641503
user4 -0.792118 -0.900149 -0.466569 -0.641503  1.000000


- itemCF：喜欢o1的人有可能也喜欢o2
- userCF：u1喜欢的物品u2可能喜欢

In [13]:
weighted_scores = 0.
corr_values_sum = 0.
target_item = 'E'
# 基于皮尔逊相关系数预测用户评分
print(similarity_matrix)
for user in sim_users:
    corr_value = similarity_matrix.loc[target_user, user]
    user_mean_rating = np.mean(list(user_data[user].values()))

    weighted_scores += corr_value * (user_data[user][target_item] - user_mean_rating)
    corr_values_sum += corr_value

other_user2target_item_rate = np.mean(list(user_data[target_user].values()))
target_item_pred = other_user2target_item_rate + weighted_scores / corr_values_sum
print(f'用户{target_user}对物品{target_item}的预测评分为：{target_item_pred}')
user_data[target_user][target_item] = target_item_pred
pprint(user_data)

          Alice     user1     user2     user3     user4
Alice  1.000000  0.852803  0.707107  0.000000 -0.792118
user1  0.852803  1.000000  0.467707  0.489956 -0.900149
user2  0.707107  0.467707  1.000000 -0.161165 -0.466569
user3  0.000000  0.489956 -0.161165  1.000000 -0.641503
user4 -0.792118 -0.900149 -0.466569 -0.641503  1.000000
用户Alice对物品E的预测评分为：4.871979899370592


NameError: name 'pprint' is not defined